### Extract data


In [36]:
import glob
import os
import pandas as pd


In [37]:
data_directory = '/home/dej/mes-projets/etl-log/data/'
# list of filepaths
file_path_list = []

# we walk through all subdirectories to find .json files
for subdir, dirs, files in os.walk(data_directory):
    for file in files:
        file_path_list.append(os.path.join(subdir, file))
print("nombre de fichier : ", len(file_path_list))

nombre de fichier :  30


In [45]:
# List of dataframes for each file
dfs = [pd.read_json(x, lines=True) for x in file_path_list]

# we concatenate all the dataframes in only one
df = pd.concat(dfs, axis=0, ignore_index=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,52,None,307,1541207073796,None,
1,None,Logged In,Celeste,F,1,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53


#### Clean of the dataframe

In [56]:
# We look after na values

df_valid = df.dropna()
df_valid.shape

(6820, 18)

#### Transform data


In [57]:
import datetime
# transform ts into week of the year
weeks = df_valid.ts.apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0).weekday())
weeks
df_valid['week'] = weeks
df_valid.shape

<ipython-input-57-58f16c4c0f0c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['week'] = weeks


(6820, 19)

In [94]:
# We divide in dataframes for each week
weeks_unique = weeks.drop_duplicates()

df_per_week = {week:df_valid[df_valid['week'] == week] for week in weeks}
# df_per_week

In [96]:
for week, df in df_per_week.items():
    df_per_week[week] = df[['week', 'song', 'sessionId']]\
        .groupby(['week', 'song'], as_index=False)\
        .agg('count')\
        .sort_values('sessionId', ascending=False)\
        .head(1)
    print(df_per_week[week].head(5))

     week     song  sessionId
420     5  Secrets          4
     week                   song  sessionId
211     6  Lucky (Album Version)          3
     week  song  sessionId
834     0  Undo          9
     week           song  sessionId
827     4  Sehr kosmisch          6
     week     song  sessionId
648     1  Revelry          6
      week            song  sessionId
1220     2  You're The One          7
      week            song  sessionId
1009     3  You're The One         10


In [105]:
best_song_per_week = pd.concat([df for df in df_per_week.values()],
                     axis=0, ignore_index=True)
best_song_per_week

,week,song,sessionId
0,5,Secrets,4
1,6,Lucky (Album Version),3
2,0,Undo,9
3,4,Sehr kosmisch,6
4,1,Revelry,6
5,2,You're The One,7
6,3,You're The One,10


In [106]:
best_song_per_week.to_csv("output.csv", header=True, index=False)

In [107]:
test = pd.read_csv("output.csv")

In [108]:
test

,week,song,sessionId
0,5,Secrets,4
1,6,Lucky (Album Version),3
2,0,Undo,9
3,4,Sehr kosmisch,6
4,1,Revelry,6
5,2,You're The One,7
6,3,You're The One,10


#### load to database

In [110]:
import sqlalchemy
USER = 'dej'
PASSWORD = 'dej'
HOST = 'localhost'
PORT = '5432'
DATABASE = 'etl_log'

conn_uri = 'postgresql://{}:{}@{}:{}/{}'.format(USER, PASSWORD, HOST, PORT, DATABASE)
db_engine = sqlalchemy.create_engine(conn_uri)

In [115]:
best_song_per_week.to_sql("best_song_per_week",
                          db_engine,
                          if_exists='replace')
